In [2]:
#Dataset
import numpy as np
import math

def sigmoid(z):
    return 1/(1 + np.exp(-z))

# initialize a random datqset with n_samples, return (x,y) of dataset
def create_dataset(n_samples: int, n_features: int, mu = 0, sigma = 1):
    weights = np.random.uniform(-10, 10, n_features + 1)
    x = np.random.normal(mu, sigma, (n_samples, n_features))
    intercept = np.ones((n_samples, 1))
    x = np.concatenate((intercept, x), axis=1)
    linear_combination = np.dot(x, weights)
    probabilities = sigmoid(linear_combination)
    y = (probabilities > 0.5).astype(int)

    return x, y

In [3]:
# logistic model class
class LogisticModel:
    def __init__(self, learning_rate: float, num_parameters: int):
        self.learning_rate = learning_rate
        self.theta = self.model_init(num_parameters)
        self.log_loss = 0
    @staticmethod
    def __sigmoid(z):
        return 1/(1 + np.exp(-z))
    @staticmethod
    def model_init(num_parameters: int):
        return np.random.normal(0, 0.1, num_parameters)
        
    @staticmethod
    def gradient(y, h_x, x):
        return (h_x - y) * x
    
    @staticmethod
    def log_loss_update(y, h): 
        h = np.clip(h, 1e-15, 1 - 1e-15)
        return -(y * math.log(h) + (1 - y) * math.log(1 - h))
    
    def h(self, x):
        return self.__sigmoid(np.dot(x, self.theta))
    
    def inference(self, x):
        probability = self.h(x)
        return 1 if probability > 0.5 else 0
    



In [13]:
# model init, training, loss functions
def model_init(num_parameters: int):
    return np.random.normal(0, 0.1, num_parameters)


def training(num_epoch: int, learning_rate: float, logistic_model: LogisticModel, mini_batch_size: int, data_points: np.ndarray,
             labels: np.ndarray):
    num_mini_batch = len(data_points) // mini_batch_size

    for i in range(num_epoch):
        # shuffle
        random_indeces = np.random.choice(len(data_points), len(data_points), replace=False)
        shuffled_data_points = data_points[random_indeces]
        shuffled_labels = labels[random_indeces]
        x_batches = np.array_split(shuffled_data_points, num_mini_batch)
        y_batches = np.array_split(shuffled_labels, num_mini_batch)
        for x_batch, y_batch in zip(x_batches, y_batches):
            update_step_value = np.zeros(logistic_model.theta.shape[0])
            for x, y in zip(x_batch, y_batch):
                h_x = logistic_model.h(x)
                update_step_value += logistic_model.gradient(y, h_x, x)
                logistic_model.log_loss += logistic_model.log_loss_update(y, h_x)
            logistic_model.theta -= learning_rate * update_step_value
        logistic_model.log_loss = logistic_model.log_loss / len(data_points)
        print(f"epoch log loss: {logistic_model.log_loss}")
        logistic_model.log_loss = 0





In [5]:
def test(logistic_model: LogisticModel, x_test: np.ndarray, y_test: np.ndarray):
    correct_predictions = 0
    for x, y in zip(x_test, y_test):
        prediction = logistic_model.inference(x)

        if prediction == y:
            correct_predictions +=1
        
    return correct_predictions / len(x_test)
        

In [14]:
# training
x_s, y_s = create_dataset(10000, 3)

split_index = int(0.8*len(x_s))
x_train, x_test = x_s[:split_index], x_s[split_index:]
y_train, y_test = y_s[:split_index], y_s[split_index:]

logistic_model = LogisticModel(0.01, x_s.shape[1])

training(2, 0.01, logistic_model, 32, x_train, y_train)




[ 0.02728101 -0.08458381 -0.12332572  0.18438048]
epoch log loss: 0.19345915289758278
epoch log loss: 0.1081370447830911
[ 3.9946352  -2.64290778  4.07206803  0.45311932]


In [16]:
#model test
logistic_model_2 = LogisticModel(0.01, x_s.shape[1])
print("Accuracy without training: ", test(logistic_model_2, x_test, y_test))
print("Accuracy with training: ", test(logistic_model, x_test, y_test))

Accuracy without training:  0.3105
Accuracy with training:  0.995
